In [1]:
import os, sys
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.append(BASE_DIR)

In [2]:
from src.models.Detection.Faster_RCNN import FasterRCNN

ImportError: cannot import name 'Mapping' from 'collections' (C:\Users\Khaliladib\anaconda3\envs\yoloc\lib\collections\__init__.py)